In [2]:
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf

# for shared metastore (shared across all users)
spark = SparkSession.builder.appName("Model Testing").config("hive.metastore.uris", "thrift://amok:9083", conf=SparkConf()).getOrCreate() \

# for local metastore (your private, invidivual database) add the following config to spark session
spark.sql("USE 2023_11_02")

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/hadoop-3.2.2/share/hadoop/common/lib/slf4j-log4j12-1.7.25.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/apache-hive-2.3.7-bin/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Log4jLoggerFactory]
2024-01-15 12:48:39,408 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-15 12:48:42,668 WARN spark.ExecutorAllocationManager: Dynamic allocation without a shuffle service is an experimental feature.


DataFrame[]

In [3]:
#import pyspark.pandas as ps
from pyspark.sql.functions import lit,col
import pandas as pd
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
#from boruta import BorutaPy
#from fredapi import Fred
from sklearn.linear_model import Lasso
from sklearn.model_selection import TimeSeriesSplit
import csv
from pyspark.sql import functions as F
from functools import reduce
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from CreateDataset import get_features_all_stocks_seq

In [4]:
df = pd.read_csv('imploded_stocks_price.csv', index_col=False)
df['Implosion_Start_Date'] = pd.to_datetime(df['Implosion_Start_Date'])
df['Implosion_End_Date'] = pd.to_datetime(df['Implosion_End_Date'])
df = get_features_all_stocks_seq(df, all_feats=False)

/opt/spark/python/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/opt/spark/python/pyspark/sql/pandas/conversion.py:331: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
2024-01-15 12:49:38,462 WARN scheduler.TaskSetManager: Stage 1 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.


238211


2024-01-15 12:49:47,755 WARN scheduler.TaskSetManager: Stage 6 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.


238211


2024-01-15 12:49:54,054 WARN scheduler.TaskSetManager: Stage 12 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.


98647


In [5]:
df.show()

2024-01-15 12:51:01,842 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
2024-01-15 12:51:03,554 WARN scheduler.TaskSetManager: Stage 22 contains a task of very large size (2969 KiB). The maximum recommended task size is 1000 KiB.


+--------+----+----------+----------------------+--------------------+--------------------+--------------------+--------------------+----------------------+--------------------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+
| fsym_id|year|      date|ff_net_inc_bef_xord_gr|     ff_capex_assets|     ff_div_yld_secs|             ff_roic|         ff_earn_yld|ff_debt_st_x_curr_port|             ff_roce|      ff_oper_inc_gr|    ff_reinvest_rate|           ff_wkcap|  ff_commiss_inc_net| ff_oper_cf_fix_chrg|  ff_debt_entrpr_val|ff_net_inc_dil_bef_unusual|             ff_xord|        ff_xord_disc|      ff_liabs_lease